# CSC413 Project: Toxicity Detector

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models, torchvision.datasets

## Data

We will begin by downloading the data onto Google Colab from kaggle. The data is retrieved from https://www.kaggle.com/datasets/fizzbuzz/cleaned-toxic-comments

In [1]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload() # Upload kaggle.json file

In [ ]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets download -d fizzbuzz/cleaned-toxic-comments
! unzip cleaned-toxic-comments.zip

cleaned-toxic-comments.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  cleaned-toxic-comments.zip
  inflating: test_preprocessed.csv   
  inflating: train_preprocessed.csv  


## Data Analysis and Processing
We will analyze the type and amount of data that we will work with and introduce a validation set.

In [20]:
import pandas as pd
df_train = pd.read_csv('/content/train_preprocessed.csv')
df_test = pd.read_csv('/content/test_preprocessed.csv')

# Randomly split the train set into a 15% validation and 85% train set
df_val = df_train.sample(n=int(len(df_train) * 0.15))
df_train = df_train.drop(df_val.index)

# First few samples in the data set of each toxicity level
print(df_train.head())
print(df_train[df_train['toxic']==1.0].head())
print(df_train[df_train['threat']==1.0].head())
print(df_train[df_train['severe_toxic']==1.0].head())

                                        comment_text                id  \
0  explanation why the edits made under my userna...  0000997932d777bf   
1  d aww  he matches this background colour i m s...  000103f0d9cfb60f   
2  hey man  i m really not trying to edit war  it...  000113f07ec002fd   
4  you  sir  are my hero  any chance you remember...  0001d958c54c6e35   
5   congratulations from me as well  use the tool...  00025465d4725e87   

   identity_hate  insult  obscene    set  severe_toxic  threat  toxic  \
0            0.0     0.0      0.0  train           0.0     0.0    0.0   
1            0.0     0.0      0.0  train           0.0     0.0    0.0   
2            0.0     0.0      0.0  train           0.0     0.0    0.0   
4            0.0     0.0      0.0  train           0.0     0.0    0.0   
5            0.0     0.0      0.0  train           0.0     0.0    0.0   

   toxicity  
0       0.0  
1       0.0  
2       0.0  
4       0.0  
5       0.0  
                                

In [21]:
training_samples = len(df_train)
val_samples = len(df_val)
testing_samples = len(df_test)
print(f'Train samples: {training_samples}')
print(f'Validation samples: {val_samples}')
print(f'Test samples: {testing_samples}')

Train samples: 135636
Validation samples: 23935
Test samples: 153164
